# Introduction

Ce projet a pour objectif d’explorer et d’analyser un jeu de données d’e-mails réels contenant à la fois des messages légitimes (Safe) et des messages frauduleux (Phishing).
Plutôt que de faire une classification supervisée, nous appliquons ici des méthodes non supervisées — notamment le clustering — pour tenter d’identifier automatiquement des regroupements naturels d’e-mails similaires.
Nous testerons plusieurs approches :
Vectorisation TF-IDF et par embeddings sémantiques
Réduction de dimension (PCA, UMAP)
Clustering (KMeans, DBSCAN, HDBSCAN)

In [ ]:
from IPython.display import display

import sys, os
sys.path.append(os.path.abspath('utils'))
from utils import *

import logging
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("numba").setLevel(logging.ERROR)
logging.getLogger("hdbscan").setLevel(logging.ERROR)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OMP_DISPLAY_ENV"] = "FALSE"
os.environ["OMP_WAIT_POLICY"] = "PASSIVE"
sys.stdout.flush()

# 🎀 Étape 1 — Compréhension et nettoyage
On commence par lire le fichier CSV contenant les e-mails et leur type (Safe ou Phishing).
Nous affichons la taille et un aperçu des premières lignes pour comprendre la structure.

In [ ]:
df = pd.read_csv('data/Phishing_Email.csv')

print(f"Nombre total de lignes : {df.shape[0]}")
df.head()

Nous identifions les colonnes inutiles (ex. Unnamed: 0) et les valeurs manquantes éventuelles.


In [ ]:
df = df.drop(columns=['Unnamed: 0'], errors='ignore')
print('Valeurs manquantes :')
print(df.isna().sum())

On calcule la longueur de chaque e-mail pour comprendre la distribution générale.
Cette étape aide à repérer les e-mails vides ou anormalement longs.

In [ ]:
df['Email Text'] = df['Email Text'].astype(str)
df['text_length'] = df['Email Text'].apply(len)

print(df['Email Type'].value_counts())
df['text_length'].describe()

Visualisons la distribution des longueurs d’e-mails selon leur type.
Cela permet de détecter la présence d’outliers visibles.

In [ ]:
plot_text_length_distribution(df, text_col="text_length", label_col="Email Type", bins=60)

Nous analysons les e-mails très courts (< 30 caractères) et très longs (> 10 000)
pour vérifier leur contenu avant toute suppression.

In [ ]:
print('--- Exemples d’e-mails très courts (<30 caractères) ---')
display(df[df['text_length'] < 30][['Email Text','Email Type']].sample(10))

print('\n--- Exemples d’e-mails très longs (>10 000 caractères) ---')
display(df[df['text_length'] > 10000][['Email Text','Email Type']].sample(30))

In [ ]:
df["text_length"].quantile([0.95, 0.98, 0.99, 0.995])

2% des emails seulement sont > 10 000 caractères

Nous décidons de :
- Supprimer les lignes vides ou marquées “empty” ou "nan"
- Conserver les e-mails courts pertinents
- Supprimer uniquement ceux > 10 000 (trop longs ou HTML massifs)

Ainsi, on garde un maximum de signal sans bruit inutile.

In [ ]:
df = df[df['Email Text'].notna()]
df = df[~df['Email Text'].str.lower().str.strip().isin(['', 'empty', 'nan'])]
df = df[df['text_length'] <= 10000]
print(f"Nouvelle taille du dataset : {df.shape[0]}")

On revérifie la distribution après nettoyage pour s'assurer
que la forme générale reste réaliste et qu'on n'a pas perdu d'information utile.

In [ ]:
plot_text_length_distribution(df, text_col="text_length", label_col="Email Type", bins=50, cleaned=True)

- Aucune valeur manquante restante 
- Équilibre des classes conservé 
- Suppression uniquement du bruit (HTML, empty, artefacts) 

Le dataset est maintenant propre et prêt pour l'analyse sémantique et le Machine Learning.

In [ ]:
print(df['Email Type'].value_counts(normalize=True)*100)
print('\nAperçu final :')
df.head()

In [ ]:
df = df.drop(columns=['text_length'], errors='ignore')

# 🎀 Étape 2 — Prétraitement NLP

Après avoir conservé le texte brut des e-mails sans nettoyage excessif, l’objectif de cette étape est d’extraire des **indicateurs mesurables** capables de traduire les irrégularités linguistiques typiques des e-mails de phishing.  

Ces indicateurs — appelés *features NLP* — transforment les anomalies structurelles et stylistiques en valeurs numériques :  
- longueur du texte,  
- ponctuation excessive,  
- majuscules,  
- symboles financiers,  
- présence de balises ou caractères HTML, etc.  

L’idée est de permettre au modèle d’apprentissage de **détecter automatiquement les schémas suspects** dans la manière dont un texte est rédigé, sans dépendre du vocabulaire lui-même.  

In [ ]:
# Application de la fonction à tout le dataset
tqdm.pandas(desc="Extraction des features NLP avancées")
nlp_features_df = df["Email Text"].progress_apply(extract_nlp_features).apply(pd.Series)

# Fusion
df_nlp = pd.concat([df, nlp_features_df], axis=1)
df_nlp.head()

In [ ]:
results = analyze_nlp_features(df_nlp, label_col="Email Type", diff_threshold=0.5, corr_threshold=0.8)

# means = results["means"]
# corr = results["corr_matrix"]
# final_features = results["final_features"]

Une première analyse exploratoire montre que les e-mails de phishing se distinguent légèrement par leur longueur et leur nombre de mots, mais les autres indicateurs lexicaux simples (ponctuation, majuscules, symboles) ne présentent pas de différences significatives.
Cela suggère que les caractéristiques de surface ne suffisent pas à discriminer efficacement les e-mails frauduleux, d’où l’intérêt d’utiliser des représentations sémantiques (embeddings) pour capturer le contenu réel des messages.

# 🎀 Étape 3 - Vectorisation du texte avec des embeddings
Les représentations basées sur les mots (*bag-of-words*, *TF-IDF*) traitent le texte comme une simple liste de termes.  
Elles ne tiennent pas compte du **sens**, du **ton**, ni du **contexte**, éléments pourtant essentiels dans la détection de phishing.

Pour aller plus loin, nous utilisons des **embeddings sémantiques**, c’est-à-dire des vecteurs numériques qui capturent la signification globale d’un message.  
Chaque e-mail est ainsi projeté dans un **espace vectoriel continu**, où :
- deux e-mails similaires dans le sens ou le ton seront proches,  
- et deux e-mails très différents (ex : “facture” vs “vérifiez votre compte”) seront éloignés.

Nous utilisons ici le modèle **`all-MiniLM-L6-v2`** de la bibliothèque `sentence-transformers`, qui offre un excellent compromis entre **vitesse** et **précision**.

In [ ]:
final_df = get_or_build_embeddings(df, text_col="Email Text", save_path="data/final_embeddings.parquet")
final_df.head()

# 🎀 Étape 4 — Réduction de dimension (UMAP, PCA, t-Sne)
L’objectif de cette étape est de visualiser la structure des données d’e-mails et de vérifier s’il existe des regroupements naturels entre les e-mails de type **Phishing** et **Safe**.  
Pour cela, deux techniques de réduction de dimension ont été utilisées : t-Sne & UMAP

In [ ]:
X = final_df.drop(columns=["Email Text", "Email Type"]).values
y = final_df["Email Type"].values

palette = get_palette()

In [ ]:
X_umap = run_umap_final(
    X=X,             
    y=y,      
    n_neighbors=30,           
    min_dist=0.1,
    metric="cosine",
    n_epochs=600,
    densmap=True,            
    cache_path="data/umap_final.npy",
    palette=palette
)

In [ ]:
X_tsne = run_tsne_final(
    X=X,
    y=y,
    perplexity=30,
    learning_rate=200,
    max_iter=1000,
    metric="cosine",
    cache_path="data/tsne_final.npy",
    palette=palette
)

### Analyse et interprétation des résultats — Réduction de dimension (UMAP + t-SNE)

#### Résultats UMAP  
L’algorithme **UMAP** (*Uniform Manifold Approximation and Projection*) a été appliqué directement sur les embeddings textuels (384 dimensions, modèle `all-MiniLM-L6-v2`) sans normalisation ni PCA préalable.  
Les paramètres retenus sont :  
- **n_neighbors = 30**  
- **min_dist = 0.1**  
- **metric = cosine**

Le score de **trustworthiness obtenu est de 0.884**, indiquant une bonne préservation des relations locales, avec une légère distorsion sur les voisinages les plus fins.  

Sur le graphique, on observe :
- Une **structure globale cohérente** où les e-mails sûrs (en bleu) se regroupent dans plusieurs zones distinctes,  
- Tandis que les e-mails de phishing (en rose) forment des **amas plus denses**, traduisant un vocabulaire plus répétitif et des thématiques restreintes (banque, livraison, promotions, etc.).  

UMAP offre ainsi une **vue d’ensemble stable** et lisible de la distribution sémantique du corpus, bien adaptée à l’analyse de la **densité et de la continuité entre sous-groupes**.

---

#### Résultats t-SNE  
L’algorithme **t-SNE** (*t-Distributed Stochastic Neighbor Embedding*) a également été appliqué directement sur les embeddings.  
Les paramètres optimaux sont :  
- **perplexity = 30**  
- **learning_rate = 200**  
- **metric = cosine**

Le score de **trustworthiness atteint 0.955**, ce qui est excellent : la projection 2D préserve plus de **95 % des relations de voisinage réelles** dans l’espace original.  

Visuellement :
- Les **deux classes** (phishing vs safe) sont **nettement mieux séparées** qu’avec UMAP,  
- Les **groupes internes** au sein des e-mails de phishing apparaissent plus clairement (ex. : “banques”, “livraisons frauduleuses”, “support technique”).  

t-SNE met donc en évidence une **structure locale fine et fiable**, au prix d’une vision globale un peu moins continue que celle d’UMAP.

---

#### Interprétation comparative  

| Méthode | Fidélité locale (*Trustworthiness*) | Structure globale | Interprétation principale |
|----------|------------------------------------|------------------|----------------------------|
| **UMAP** | 0.884 | Bonne continuité globale, clusters denses | Vision d’ensemble fluide, structure thématique identifiable |
| **t-SNE** | 0.955 | Moins globale mais très précise localement | Excellente séparation et découverte de sous-groupes |

Ces résultats montrent que :
- **t-SNE** est plus adapté à la **visualisation précise** des familles de phishing,  
- tandis qu’**UMAP** conserve mieux la **structure globale** du corpus et les **liens entre thématiques voisines**.

---

#### Conclusion  
Les deux projections confirment une **séparation sémantique claire** entre les e-mails sûrs et les e-mails de phishing.  
- **UMAP** offre une représentation fluide et connectée, idéale pour l’exploration.  
- **t-SNE** fournit une vue détaillée et très fidèle, avec une **trustworthiness exceptionnelle (0.955)**.  

Ces représentations serviront de **base pour l’étape de clustering non supervisé** (HDBSCAN, KMeans) afin d’identifier automatiquement les **types de phishing** présents dans le corpus.

# 🎀 Étape 5 — Clustering non supervisé (DBSCAN, HDBSCAN, KMEANS)

## Sur t-Sne

In [ ]:
results_hdb, best_hdb, labels_hdb = explore_hdbscan(X_tsne)
summary_hdb = summarize_clusters(final_df, labels_hdb)
display_top_words(df_clusters=summary_hdb["df_clusters"], summary=summary_hdb["summary"])

In [ ]:
metrics = evaluate_clustering(summary_hdb["summary"], summary_hdb["df_clusters"])

In [ ]:
k_final = 45
print(f"Utilisation de k={k_final}")

kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=20)
labels_km = kmeans_final.fit_predict(X_tsne)

plt.figure(figsize=(7, 6))
sns.scatterplot(
        x=X_tsne[:, 0],
        y=X_tsne[:, 1],
        hue=labels_km,
        palette="Spectral",
        s=20,
        alpha=0.85,
        legend=False
)
plt.title(
        f"KMeans — 45 clusters \n",
        color="#e75480"
)

plt.tight_layout()
plt.show()

summary_km = summarize_clusters(final_df, labels_km)

display_top_words(summary_km["df_clusters"], summary_km["summary"])


In [ ]:
preview_cluster_emails(summary_km["df_clusters"], cluster_id=11, n=20)

In [ ]:
df_dbscan, best_dbscan, labels_best = explore_dbscan(X_tsne)
summary_db = summarize_clusters(final_df, labels_best)
display_top_words(df_clusters=summary_db["df_clusters"], summary=summary_db["summary"])

faire analyse cluster pour dbscan et kmeans ??

# 🎀 Étape 6 : Analyse sémantique des types de clusters (avec LLM)

In [ ]:
analyze_phishing_clusters_individual(
    summary_hdb["df_clusters"],
    summary_hdb["summary"],
    algorithm_name="hdbscan",   
    n_per_cluster=40,           
    model="mistral:7b-instruct",
    max_retries=3,              
    cooldown=4                 
)

In [ ]:
preview_cluster_emails(summary_hdb["df_clusters"], cluster_id=14, n=40, max_chars=200)

In [ ]:
df_llm_hdb = load_llm_results("data/phishing_categories/hdbscan")
print(f"{len(df_llm_hdb)} fichiers de clusters chargés.")
print(df_llm_hdb)

In [ ]:
print("X_emb:", X_tsne.shape)
print("df_clusters:", summary_hdb["df_clusters"].shape)
print("df_llm:", df_llm_hdb.shape)

In [ ]:
df_enriched_hdb = summary_hdb["df_clusters"].merge(
    df_llm_hdb,
    on="cluster",
    how="left"
)

In [ ]:
summary_hdb["df_clusters"].shape

In [ ]:
df_enriched_hdb.shape

In [ ]:
plot_categories_llm(
    X_emb=X_tsne,     
    df_enriched=df_enriched_hdb,
    name="HDBSCAN"
)

In [ ]:
df_enriched_hdb["category"].value_counts(normalize=True).round(2)

In [ ]:
df_enriched_hdb.groupby("category")["confidence"].mean().sort_values()

In [ ]:
analyze_phishing_clusters_individual(
    summary_km["df_clusters"],
    summary_km["summary"],
    algorithm_name="kmeans",   
    n_per_cluster=40,           
    model="mistral:7b-instruct",
    max_retries=0,              
    cooldown=4                  
)

In [ ]:
analyze_phishing_clusters_individual(
    summary_db["df_clusters"],
    summary_db["summary"],
    algorithm_name="dbscan",   
    n_per_cluster=40,           
    model="mistral:7b-instruct",
    max_retries=3,              
    cooldown=4                  
)

In [ ]:
df_llm_db = load_llm_results("data/phishing_categories/dbscan")
df_enriched_db = summary_db["df_clusters"].merge(
    df_llm_db,
    on="cluster",
    how="left"
)

In [ ]:
df_llm_db.head()

In [ ]:
plot_categories_llm(
    X_emb=X_tsne,     
    df_enriched=df_enriched_db,
    name="DBSCAN"
)

In [ ]:
df_llm_km = load_llm_results("data/phishing_categories/kmeans")
print(df_llm_km)

# 🎀 Étape 7 : Classification supervisée

In [ ]:
results_df, best_model = run_supervised_classification(X, y)